## Requirements

In [241]:
#Requirement Library
#a library that for math.el and logical and vectorizational operations
!pip install numpy 
#a library for process bar 
!pip install tqdm 

In [242]:
#  from time import sleep
#libraries
import numpy as np
from tqdm import tqdm
import random


## Waterman-Smith Algorithm implementation

In [292]:
#functions
def fill_matrix(H, s_A, s_B, match= 3, mis_match= -3, gap= -2): 
    #a method to generate score matrix
    y,x = H.shape
    left, up, diagonel, score = 0, 0, 0, 0 
    
    for i in range(1, y):
        for j in range(1, x):
            left = H[i][j-1] + gap
            up = H[i-1][j] + gap
            if(s_A[j-1] == s_B[i-1]):
                diagonel = H[i-1][j-1] + match
            else:
                diagonel = H[i-1][j-1] + mis_match
            score = max(left, up,diagonel)
            if(score>0):
                H[i][j] = score
    return H

def traceback(H,s_A,s_B):
    #a method that to find out maximum similar text
    result = []
    i,j = np.unravel_index(H.argmax(), H.shape)
    up, left, diagonal, maxx = 0, 0, 0, 0
    while(H[i][j]!=0):
        if(s_A[j-1] == s_B[i-1]):
            result.append(s_A[j-1])
        else:
            result.append("_")
        left = H[i][j-1]
        up = H[i-1][j]
        diagonal = H[i-1][j-1]
        maxx= max(left,up,diagonal)
        if(diagonal == maxx):
            i+=-1
            j+=-1
        elif(maxx==left):
            j+=-1
        else:
            i+=-1
    result.reverse()
    if(result[0] == "_"):
        result.pop(0)
    return " ".join(result)

def smith_waterman_algorithm(string1, string2, show_matrix = False, match=3, mis_match=-3, gap=-2):
    #an algorithm works with that method
    s_A = string1.split(" ")#sequence A
    s_B = string2.split(" ")#sequence B
    H = np.zeros((len(s_B)+1,len(s_A)+1), dtype = np.ushort) #unsigned integter to memory issues
    fill_matrix(H,s_A, s_B)
    if(show_matrix):
        print(H)
    return traceback(H,s_A,s_B)

def compare(string1, string2, show_matrix = False):
    matched_str = smith_waterman_algorithm(string1, string2, show_matrix = show_matrix)
    matched_txt_similarity_rate = (len(matched_str.split(" "))*2)/(len(string1.split(" "))+len(string2.split(" ")))
    return matched_str, matched_txt_similarity_rate



def runHW():
    global total_txt_count
    global H_similarity_matrix
    global H_matched_txt_matrix
    similarity = 0.0
    matched_txt = ""
    str1 = ""
    str2 = ""
    for i in tqdm(range(total_txt_count)):
        str1 = open("sources/"+str(i)+".txt", mode='r', encoding="ISO-8859-1").read()
        for j in range(i+1,total_txt_count):
            str2 = open("sources/"+str(j)+".txt", mode='r', encoding="ISO-8859-1").read()
            matched_txt , similarity = compare(str1, str2)
            H_similarity_matrix[i][j] = similarity
            H_matched_txt_matrix[i,j] = matched_txt
            


In [311]:
##run simple example with only 5 file
total_txt_count = 5
H_similarity_matrix = np.zeros((total_txt_count,total_txt_count), dtype = float) 
H_matched_txt_matrix = np.empty((total_txt_count, total_txt_count), dtype=object)

runHW()

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:05<00:00,  1.03s/it]


In [294]:
print(H_matched_txt_matrix[1][4])
print(H_matched_txt_matrix[0][1])

 _            _        _ _ _ _ _ _ _ _ _ _      _                       
about and describing


In [295]:
H_similarity_matrix # a matrix that shows file's pair similarity

array([[0.        , 0.00966184, 0.01489758, 0.00512821, 0.03041825],
       [0.        , 0.        , 0.01408451, 0.00475059, 0.21543986],
       [0.        , 0.        , 0.        , 0.00593472, 0.02536998],
       [0.        , 0.        , 0.        , 0.        , 0.00613497],
       [0.        , 0.        , 0.        , 0.        , 0.        ]])

In [319]:
#intermediate example with 20 file
total_txt_count = 20
H_similarity_matrix = np.zeros((total_txt_count,total_txt_count), dtype = float) 
H_matched_txt_matrix = np.empty((total_txt_count, total_txt_count), dtype=object)

runHW()

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [01:38<00:00,  4.90s/it]


In [322]:
#find most similar files pair
i,j = np.unravel_index(H_similarity_matrix.argmax(), H_similarity_matrix.shape)

print(str(i)+".txt and "+str(j)+".txt" + " are have most similar text")

print("Matched text is: " +  H_matched_txt_matrix[i][j] +
      " with " + str(H_similarity_matrix[i][j]) + " score")


7.txt and 18.txt are have most similar text
Matched text is: as substantially in contrast to statistics for Wikipedia in other languages (non-English 
Wikipedia). The Economist reported that the number of contributors with an average of five or more 
edits per month was relatively constant since 2008 for Wikipedia in other languages at approximately
 42,000 editors within narrow seasonal variances of about 2,000 editors up or down. The number of 
active editors in English Wikipedia, by "sharp" comparison, was cited as peaking in 2007 at 
approximately 50,000 and dropping to 30,000 by the start of with 0.4148148148148148 score


# Some Score Matrix examples

In [323]:
smith_waterman_algorithm("G C C T C N T C C C G G C", "G G G C N T C C G G G F H", show_matrix = True)

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  3  1  0  0  0  0  0  0  0  0  3  3  1]
 [ 0  3  1  0  0  0  0  0  0  0  0  3  6  4]
 [ 0  3  1  0  0  0  0  0  0  0  0  3  6  4]
 [ 0  1  6  4  2  3  1  0  3  3  3  1  4  9]
 [ 0  0  4  3  1  1  6  4  2  1  1  0  2  7]
 [ 0  0  2  1  6  4  4  9  7  5  3  1  0  5]
 [ 0  0  3  5  4  9  7  7 12 10  8  6  4  3]
 [ 0  0  3  6  4  7  6  5 10 15 13 11  9  7]
 [ 0  3  1  4  3  5  4  3  8 13 12 16 14 12]
 [ 0  3  1  2  1  3  2  1  6 11 10 15 19 17]
 [ 0  3  1  0  0  1  0  0  4  9  8 13 18 16]
 [ 0  1  0  0  0  0  0  0  2  7  6 11 16 15]
 [ 0  0  0  0  0  0  0  0  0  5  4  9 14 13]]


'G G G C C _ C N T C C C G G'

In [324]:
smith_waterman_algorithm("A B C D", "D D C D", show_matrix = True)

[[0 0 0 0 0]
 [0 0 0 0 3]
 [0 0 0 0 3]
 [0 0 0 3 1]
 [0 0 0 1 6]]


'C D'

In [325]:
smith_waterman_algorithm("f k j d a l f a ; d s k f j ; a f", "f a d l s k j ; f a s d j l ; k f s d a l f j ", show_matrix = True)

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  3  1  0  0  0  0  3  1  0  0  0  0  3  1  0  0  3]
 [ 0  1  0  0  0  3  1  1  6  4  2  0  0  1  0  0  3  1]
 [ 0  0  0  0  3  1  0  0  4  3  7  5  3  1  0  0  1  0]
 [ 0  0  0  0  1  0  4  2  2  1  5  4  2  0  0  0  0  0]
 [ 0  0  0  0  0  0  2  1  0  0  3  8  6  4  2  0  0  0]
 [ 0  0  3  1  0  0  0  0  0  0  1  6 11  9  7  5  3  1]
 [ 0  0  1  6  4  2  0  0  0  0  0  4  9  8 12 10  8  6]
 [ 0  0  0  4  3  1  0  0  0  3  1  2  7  6 10 15 13 11]
 [ 0  3  1  2  1  0  0  3  1  1  0  0  5 10  8 13 12 16]
 [ 0  1  0  0  0  4  2  1  6  4  2  0  3  8  7 11 16 14]
 [ 0  0  0  0  0  2  1  0  4  3  1  5  3  6  5  9 14 13]
 [ 0  0  0  0  3  1  0  0  2  1  6  4  2  4  3  7 12 11]
 [ 0  0  0  3  1  0  0  0  0  0  4  3  1  2  7  5 10  9]
 [ 0  0  0  1  0  0  3  1  0  0  2  1  0  0  5  4  8  7]
 [ 0  0  0  0  0  0  1  0  0  3  1  0  0  0  3  8  6  5]
 [ 0  0  3  1  0  0  0  0  0  1  0  0  3  1  1  6  5  3]
 [ 0  3  1  0  0  0  0  3  1  0

'f a _ d _ s k _ j ; _ f'

## Extra part

#### -I also implement Waterman-Smith algorithm with only recursion algorithm 

#### -I also coded script to parse and text processing for converting pdf

#### -I also added progress bar to know how long process's duration

In [332]:
#recurison version  Waterman-Smith algorithm
def fill_matrixR(H, s_A, s_B, j, i, match= 3, mis_match= -3, gap= -2): #current_location_x 
        if (j == (len(s_A)+1) and i == (len(s_B))):
            return H
        elif (j == (len(s_A))+1):
            return fill_matrixR(H, s_A, s_B, 1, i+1,match, mis_match,gap)
        else:
            if(s_A[j-1]==s_B[i-1]):
                diagonal = H[i-1][j-1] + match
            else:
                diagonal = H[i-1][j-1] + mis_match
                
            up = H[i-1][j] + gap
            left = H[i][j-1] + gap           
            result = max(diagonal,up,left)
            if (result<0):
                H[i][j] = 0
            else:
                H[i][j] = result
            return fill_matrixR(H, s_A, s_B, j+1, i, match, mis_match, gap)
        
def tracebackR(H,string,s_A,s_B,i,j,add=True):
    if(H[i][j] == 0):
        string
        return string
    else:
        if(s_A[j-1] == s_B[i-1]):
            string+=s_A[j-1]
        elif(H[i][j]!=0):
            string+="-"
        up = H[i-1][j]
        left = H[i][j-1]
        diagonel = H[i-1][j-1]
        maxx =max(up,left,diagonel) 
        if(maxx == up):
            return tracebackR(H, string, s_A,s_B, i-1,j,False)
        elif(maxx == left):
            return tracebackR(H, string, s_A,s_B, i,j-1,False)
        else:
            return tracebackR(H, string, s_A,s_B, i-1,j-1,True)

def smith_waterman_algorithmR(s_A,s_B, match = 3, mis_match=-3, gap=-2, show_matrix = False):
    H = np.zeros((len(s_B)+1,len(s_A)+1), dtype = int)
    
    fill_matrixR(H, s_A, s_B, 1, 1, match, mis_match, gap)

    i,j = np.unravel_index(H.argmax(), H.shape)
    if(show_matrix):
        print(H)
    result=tracebackR(H,"",s_A,s_B,i,j)[::-1]
    if(result[0] == '-'):
        return result[1:]
    return result  

In [335]:
smith_waterman_algorithmR("yine yaz kalemim ile", "kale yaziyorum yine ve", show_matrix = True)

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  3  1  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  3  1  0  1  6  4  2  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  0  0  0  4  9  7  5  3  1  0  0  3  1]
 [ 0  0  0  0  3  1  0  0  0  0  0  2  7 12 10  8  6  4  2  1  6]
 [ 0  0  0  0  1  6  4  2  0  3  1  0  5 10  9  7  5  9  7  5  4]
 [ 0  3  1  0  0  4  9  7  5  3  1  0  3  8  7  6  4  7  6  4  2]
 [ 0  1  0  0  0  2  7 12 10  8  6  4  2  6  5  4  3  5  4  3  1]
 [ 0  0  0  0  0  0  5 10 15 13 11  9  7  5  3  2  1  3  2  1  0]
 [ 0  0  3  1  0  0  3  8 13 12 10  8  6  4  2  6  4  2  6  4  2]
 [ 0  3  1  0  0  0  3  6 11 10  9  7  5  3  1  4  3  1  4  3  1]
 [ 0  1  0  0  0  0  1  4  9  8  7  6  4  2  0  2  1  0  2  1  0]
 [ 0  0  0  0  0  0  0  2  7  6  5  4  3  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  5  4  3  2  1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  3  2  1  0  0  0  3  1  3  1  0  0  0]
 [ 0  0  0

'e yaz'

In [336]:
smith_waterman_algorithmR("hala nieye", "halama bayrama gidiyorum", show_matrix = True)

[[ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  3  1  0  0  0  0  0  0  0  0]
 [ 0  1  6  4  3  1  0  0  0  0  0]
 [ 0  0  4  9  7  5  3  1  0  0  0]
 [ 0  0  3  7 12 10  8  6  4  2  0]
 [ 0  0  1  5 10  9  7  5  3  1  0]
 [ 0  0  3  3  8  7  6  4  2  0  0]
 [ 0  0  1  1  6 11  9  7  5  3  1]
 [ 0  0  0  0  4  9  8  6  4  2  0]
 [ 0  0  3  1  3  7  6  5  3  1  0]
 [ 0  0  1  0  1  5  4  3  2  6  4]
 [ 0  0  0  0  0  3  2  1  0  4  3]
 [ 0  0  3  1  3  1  0  0  0  2  1]
 [ 0  0  1  0  1  0  0  0  0  0  0]
 [ 0  0  3  1  3  1  0  0  0  0  0]
 [ 0  0  1  0  1  6  4  2  0  0  0]
 [ 0  0  0  0  0  4  3  1  0  0  0]
 [ 0  0  0  0  0  2  1  6  4  2  0]
 [ 0  0  0  0  0  0  0  4  3  1  0]
 [ 0  0  0  0  0  0  0  3  1  0  0]
 [ 0  0  0  0  0  0  0  1  0  4  2]
 [ 0  0  0  0  0  0  0  0  0  2  1]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]]


'hala'

In [ ]:
#script to parsing and process pdf to txt
import sys
import codecs
import PyPDF2
from tqdm import tqdm


count = 0
def translateTotxt(file_name):
    global count
    # creating a pdf file object 
    pdfFileObj = open(file_name  +".pdf", 'rb') 
    file1 = open(file_name + ".txt","w")


    # creating a pdf reader object 
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 

    num_pages = pdfReader.numPages
    print("num_pages")
    extracted_str = ""
    j = 0
    for i in range(int(num_pages)):
        j+=1
        if(j==5):
            file1= codecs.open(str(count) + ".txt", "w", "utf-8")
            file1 = open(str(count) + ".txt","w")
            try:
                if(len(extracted_str)>2000):
                    extracted_str = extracted_str[:2000] #turnicate
                
                print(extracted_str)
                file1.write(extracted_str)
            except:
                print("there is an except")
                count=-1
            extracted_str = ""
            file1.close()
            count+=1
            j=0
    extracted_str += pdfReader.getPage(i).extractText()
    print(len(extracted_str))
    file1.close()
    pdfFileObj.close()


for i in tqdm(range(1, 3)):
    translateTotxt("a"+str(i))